In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as plx

from stooq import *
import json


In [2]:
# Altere estas variáveis caso os arquivos não tenham sido encontrados
stooq_dir = 'data/stooq'
fundamentals_dir = 'data/sec/companyfacts'
tickers_path = 'data/sec/company_tickers.json'

In [3]:
period = 'daily'
market = 'us'
sample_companies = [
    'MSFT.US',
    'AVGO.US',
    'NVDA.US',
    'TSM.US',
    'TSLA.US',
    'META.US',
    'IBM.US',
    'AAPL.US',
    'ASML.US',
    'AMZN.US',
    'GOOG.US',
    'LLY.US',
    'WMT.US',
    'JPM.US',
    'V.US',
    'UNH.US',
    'XOM.US',
    'ORCL.US',
    'MA.US'
]

In [4]:
cik_by_ticker = None
with open(tickers_path, 'r') as file:
    cik_by_ticker = json.load(file)
cik_by_ticker = { value['ticker'] + '.US': value['cik_str'] for _key, value in cik_by_ticker.items() }

In [5]:
import av_api as av
import json
import os

In [6]:
ALPHA_VANTAGE_KEY = os.environ['ALPHA_VANTAGE_KEY']
api = av.AlphaVantage(ALPHA_VANTAGE_KEY)

In [7]:
company = 'LLY'

In [8]:
#av_response = api.get_earnings(symbol=company)

In [9]:
#earnings = json.loads(av_response)
#earnings = pd.DataFrame(earnings['quarterlyEarnings'])
#earnings.columns = ['period_end', 'date', 'eps', 'estimated_eps', 'surprise', 'surprise_pct', 'report_time']
#earnings.set_index('date', inplace=True)
#earnings.sort_index(inplace=True)
#earnings['eps_ms'] = earnings['eps'].rolling(4).sum()
#earnings.index = pd.to_datetime(earnings.index)
#earnings.to_parquet(f'data/alpha_vantage/{company.lower()}_earnings.parquet')

In [10]:
earnings = pd.read_parquet(f'data/alpha_vantage/{company.lower()}_earnings.parquet')

In [11]:
earnings

,period_end,eps,estimated_eps,surprise,surprise_pct,report_time,eps_ms
date,,,,,,,
1996-04-15,1996-03-31,0.36,0.36,0,0,pre-market,NaN
1996-07-16,1996-06-30,0.32,0.32,0,0,pre-market,NaN
1996-10-22,1996-09-30,0.38,0.32,0.06,18.75,pre-market,NaN
1997-01-27,1996-12-31,0.34,0.34,0,0,pre-market,1.40
1997-04-21,1997-03-31,0.41,0.4,0.01,2.5,pre-market,1.45
...,...,...,...,...,...,...,...
2023-08-08,2023-06-30,2.11,1.98,0.13,6.5657,pre-market,7.80
2023-11-02,2023-09-30,0.1,-0.13,0.23,176.9231,pre-market,5.92
2024-02-06,2023-12-31,2.49,2.22,0.27,12.1622,pre-market,6.32


In [12]:
prices = build_prices_table(stooq_dir, period, market)
add_one_cent_noise(prices)

,<TICKER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
6436603,A.US,1999-11-18,0,29.5594,32.4842,25.9889,28.592713,6.886678e+07
6436604,A.US,1999-11-19,0,27.8972,27.9371,25.8613,26.238817,1.677358e+07
6436605,A.US,1999-11-22,0,26.8370,28.5858,26.0278,28.593530,7.242576e+06
6436606,A.US,1999-11-23,0,27.6102,28.3377,25.9889,25.993458,6.579458e+06
6436607,A.US,1999-11-24,0,26.0637,27.2445,25.9889,26.683662,5.332648e+06
...,...,...,...,...,...,...,...,...
13848043,ZZZ.US,2024-05-24,0,25.0800,25.1784,25.0800,25.186521,4.060000e+02
13848044,ZZZ.US,2024-05-28,0,25.0000,25.0100,25.0000,25.014742,2.769000e+03
13848045,ZZZ.US,2024-05-29,0,24.8500,24.8500,24.8359,24.843328,8.910000e+02
13848046,ZZZ.US,2024-05-30,0,24.9700,24.9850,24.8544,24.856840,1.127000e+03


In [13]:
#prices = pd.read_parquet('data/stooq/5 min/us/all_stocks_processed.parquet')
prices.columns = [col[1:-1].lower() for col in prices.columns]
prices = prices[ prices['ticker'] == f'{company}.US' ].copy()

In [14]:
prices['date'] = pd.to_datetime(prices['date'].astype(str))
#prices['time'] = pd.to_timedelta(prices['time'].astype(str).apply(lambda x: f'{x[0:2]}:{x[2:4]}:{x[4:6]}'))
#prices['datetime'] = prices.date + prices.time
prices.drop(columns=['time'], inplace=True)
prices.set_index('date', inplace=True)

In [15]:
prices

,ticker,open,high,low,close,vol
date,,,,,,
1982-01-04,LLY.US,2.44706,2.45420,2.42597,2.450802,1.285735e+06
1982-01-05,LLY.US,2.39733,2.43270,2.34790,2.381197,7.330994e+05
1982-01-06,LLY.US,2.34790,2.39040,2.29818,2.332241,1.847402e+06
1982-01-07,LLY.US,2.31938,2.33364,2.29818,2.336125,7.939967e+05
1982-01-08,LLY.US,2.34790,2.35504,2.32660,2.355775,8.458816e+05
...,...,...,...,...,...,...
2024-05-24,LLY.US,808.00000,810.12000,804.15000,807.431277,1.778120e+06
2024-05-28,LLY.US,809.00000,809.00000,788.25000,807.869650,2.483454e+06
2024-05-29,LLY.US,803.00000,815.61000,802.09000,811.530276,2.380041e+06


In [14]:
prices = prices.merge(earnings, 'outer', left_index=True, right_index=True)
prices['eps_ms'] = prices['eps_ms'].ffill()

In [15]:
final_df = prices[['close', 'eps_ms']].dropna()
final_df['log_adj_ey'] = np.log(1 + final_df['eps_ms']/final_df['close'].shift())
final_df.replace([-np.inf, np.inf], np.nan, inplace=True)
final_df.dropna(inplace=True)

In [16]:
final_df['log_close'] = np.log(final_df['close'])
final_df['log_return'] = np.log(final_df['close']) - np.log(final_df['close']).shift()

In [17]:
interest_rate = pd.read_csv(f'{stooq_dir}/daily/macro/us/inrtus.m.txt')
interest_rate = interest_rate[['<DATE>', '<CLOSE>']].copy()

In [18]:
interest_rate.rename(columns={'<DATE>': 'date', '<CLOSE>': 'interest'}, inplace=True) 
interest_rate['date'] = pd.to_datetime(interest_rate['date'].astype(str))
interest_rate.set_index('date', inplace=True)
interest_rate.sort_index(inplace=True)
final_df = final_df.join(interest_rate, how='outer')
final_df['interest'] = final_df['interest'].ffill()

In [19]:
final_df['log_interest'] = np.log((final_df['interest'] / 100) + 1)

In [20]:
final_df.dropna(inplace=True)

In [21]:
#for i in range(1, 10)log_adj:
#    final_df[f'log_return_l{i}'] = final_df['log_close'].shift() - final_df['log_close'].shift(2**i)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import r2_score, accuracy_score

In [23]:
#final_df['log_adj_ey_l1'] =  final_df[f'log_adj_ey'].shift()
#for i in range(2, 32):
#    final_df[f'log_adj_ey_l{i}'] =  final_df[f'log_adj_ey_l{i-1}'].shift()

In [24]:
final_df.dropna(inplace=True)

In [25]:
final_df.columns

Index(['close', 'eps_ms', 'log_adj_ey', 'log_close', 'log_return', 'interest',
       'log_interest'],
      dtype='object')

In [64]:
#X = pd.concat([final_df.iloc[:, -30:], final_df['log_adj_ey']], axis='columns')
X = final_df.drop(columns=['close', 'eps_ms', 'log_close', 'log_return', 'interest', 'log_interest'])
y = final_df['log_return']

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)

In [28]:
model = DecisionTreeClassifier(min_samples_leaf=16).fit(X_train, y_train)

In [29]:
accuracy_score(y_train, model.predict(X_train))

0.6585213032581454

In [30]:
accuracy_score(y_test, model.predict(X_test))

0.45112781954887216

In [31]:
ccp_path = model.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = ccp_path.ccp_alphas
train_scores = []
test_scores = []
for alpha in ccp_alphas:
    model_i = DecisionTreeClassifier(ccp_alpha=alpha).fit(X_train, y_train)
    train_scores.append(accuracy_score(y_train, model_i.predict(X_train)))
    test_scores.append(accuracy_score(y_test, model_i.predict(X_test)))

In [32]:
ccp_df = pd.DataFrame({
    'train_score': train_scores,
    'test_score': test_scores,
}, index=ccp_alphas)

In [33]:
plx.line(
    data_frame=ccp_df.melt(ignore_index=False).reset_index(names=['ccp_alpha']),
    #title='Cummulative returns - Model vs Buy and hold',
    x='ccp_alpha',
    y='value',
    color='variable',
    height=720,
    width=1280,
)

/home/pedro/.local/lib/python3.10/site-packages/plotly/express/_core.py:1979: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [55]:
import statsmodels.formula.api as smf
from statsmodels.tsa.arima.model import ARIMA

In [66]:
model = ARIMA(y_train, order=(1, 0, 1)).fit()
model.summary()

/home/pedro/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/pedro/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/pedro/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:             log_return   No. Observations:                 1596
Model:                 ARIMA(1, 0, 1)   Log Likelihood                4237.372
Date:                Thu, 17 Oct 2024   AIC                          -8466.745
Time:                        19:52:58   BIC                          -8445.244
Sample:                             0   HQIC                         -8458.760
                               - 1596                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0009      0.000      2.043      0.041    3.53e-05       0.002
ar.L1         -0.6315      0.072     -8.766      0.000      -0.773      -0.490
ma.L1          0.5455      0.077      7.050      0.000       0.394       0.697
sigma2         0.0003   3.93e-06     73.616      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.02   Jarque-Bera (JB):             10339.28
Prob(Q):                              0.90   Prob(JB):                         0.00
Heteroskedasticity (H):               3.19   Skew:                             0.58
Prob(H) (two-sided):                  0.00   Kurtosis:                        15.41
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [42]:
#model = DecisionTreeClassifier(ccp_alpha=0.002).fit(X_train, y_train)

In [67]:
results_test = pd.DataFrame({
    'real': y_test,
    'predicted': model.predict(X_test),
})

KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'

In [44]:
results_test = results_test.join(final_df['log_return'], how='left')

In [45]:
results_test

,real,predicted,log_return
date,,,
2022-04-20,False,False,-0.013871
2022-04-21,False,False,-0.011351
2022-04-22,False,False,-0.038288
2022-04-25,True,False,0.022938
2022-04-26,False,False,-0.008335
...,...,...,...
2024-05-24,False,False,-0.001262
2024-05-28,True,False,0.000537
2024-05-29,True,False,0.004532


In [46]:
results_test['model_return'] = ((results_test.predicted).astype(int) * results_test.log_return).cumsum()
results_test['cumm_return'] = results_test['log_return'].cumsum()

In [47]:
results_test.predicted.value_counts()

predicted
False    532
Name: count, dtype: int64

In [48]:
plx.line(
    data_frame=results_test[['model_return', 'cumm_return']].melt(ignore_index=False).reset_index(names=['date']),
    #title='Cummulative returns - Model vs Buy and hold',
    x='date',
    y='value',
    color='variable',
    height=720,
    width=1280,
)

/home/pedro/.local/lib/python3.10/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/home/pedro/.local/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

